In [69]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [70]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 869720565170082940
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 6984834037202208272
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 9800380516810853674
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 14648777152
locality {
  bus_id: 1
  links {
  }
}
incarnation: 17476237497637734064
physical_device_desc: "device: 0, name: Tesla T4, pci bus id: 0000:00:04.0, compute capability: 7.5"
]


In [71]:
import pandas as pd

# train = pd.read_csv('/content/gdrive/My Drive/twitter misinfo/new_data/train.tsv', sep = '\t')
# test = pd.read_csv('/content/gdrive/My Drive/twitter misinfo/new_data/test.tsv', sep = '\t')
# dev = pd.read_csv('/content/gdrive/My Drive/twitter misinfo/new_data/dev.tsv', sep = '\t')

# print(len(train))
# print(len(test))
# print(len(dev))
# train.head()

In [72]:
# train.dropna()
# dev.dropna()
# test.dropna()
# print(len(train))
# print(len(test))
# print(len(dev))

In [73]:
# train.head()

In [74]:
# len(train['tweet'][9001])

In [75]:
# def find_empty(df):
#     empty = list()
#     for ind in train.index:
#         sentence = train['tweet'][ind]
#         try:
#             if len(sentence) == 0:
#                 empty.append(ind)
#         except Exception as e:
#             print(sentence)
    
#     return empty

In [76]:
# empty = find_empty(test)
# print(empty)
# empty = find_empty(dev)
# print(empty)

In [77]:
def find_nan(df):
    nan_rows = list()
    for ind in df.index:
        sentence = str(df['tweet'][ind])
        try:
            if sentence == 'nan':
                nan_rows.append(ind)
        except Exception as e:
            print(sentence)
        
    return nan_rows

# nan_train = list()
# nan_train = find_nan(train)
# nan_test = find_nan(test)
# nan_dev = find_nan(dev)

# print("Train : \n", nan_train)
# print("Test : \n", nan_test)
# print("Dev : \n", nan_dev)

In [78]:
# from pandas.core.frame import DataFrame

# def df_to_tsv(dataf, some_list):
#     for i in some_list:
#         dataf.drop(i, inplace=True)
#     # df = DataFrame(self)
#     # result is only a string if no path provided, otherwise None
#     dataf.to_csv('/content/gdrive/My Drive/twitter misinfo/prepro_data/dev.tsv', sep='\t')

# # df_to_tsv(train, nan_train)
# # df_to_tsv(dev, nan_dev)

In [79]:
# df_to_tsv(train, nan_train)

In [80]:
# df_to_tsv(dev, nan_dev)

In [81]:
train = pd.read_csv('/content/gdrive/My Drive/twitter misinfo/prepro_data/train.tsv', sep = '\t')
test = pd.read_csv('/content/gdrive/My Drive/twitter misinfo/prepro_data/test.tsv', sep = '\t')
dev = pd.read_csv('/content/gdrive/My Drive/twitter misinfo/prepro_data/dev.tsv', sep = '\t')

In [82]:
nan_train = list()
nan_train = find_nan(train)
nan_test = find_nan(test)
nan_dev = find_nan(dev)

print("Train : \n", nan_train)
print("Test : \n", nan_test)
print("Dev : \n", nan_dev)

Train : 
 []
Test : 
 []
Dev : 
 []


In [83]:
# import os

# def split(filehandler, delimiter='\t', row_limit=3000,
#           output_name_template='output_%s.tsv', output_path='/content/gdrive/My Drive/twitter misinfo/new_data/', keep_headers=True):
#     import csv
#     reader = csv.reader(filehandler, delimiter=delimiter)
#     current_piece = 1
#     current_out_path = os.path.join(
#         output_path,
#         output_name_template % current_piece
#     )
#     current_out_writer = csv.writer(open(current_out_path, 'w'), delimiter=delimiter)
#     current_limit = row_limit
#     if keep_headers:
#         headers = next(reader)
#         current_out_writer.writerow(headers)
#     for i, row in enumerate(reader):
#         if i + 1 > current_limit:
#             current_piece += 1
#             current_limit = row_limit * current_piece
#             current_out_path = os.path.join(
#                 output_path,
#                 output_name_template % current_piece
#             )
#             current_out_writer = csv.writer(open(current_out_path, 'w'), delimiter=delimiter)
#             if keep_headers:
#                 current_out_writer.writerow(headers)
#         current_out_writer.writerow(row)


# split(open('/content/gdrive/My Drive/twitter misinfo/new_data/2nd.tsv', 'r'));

In [84]:
# import csv

# # read tab-delimited file
# with open('/content/gdrive/My Drive/twitter misinfo/new_data/output_2.tsv','r') as fin:
#     cr = csv.reader(fin, delimiter='\t')
#     filecontents = [line for line in cr]

# # write comma-delimited file (comma is the default delimiter)
# with open('/content/gdrive/My Drive/twitter misinfo/new_data/output_2.csv','w') as fou:
#     cw = csv.writer(fou, quotechar='', quoting=csv.QUOTE_NONE, lineterminator = '\n', escapechar = '\\')
#     cw.writerows(filecontents)


In [85]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F


class BiGRUSentiment(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, label_size, use_gpu, batch_size, dropout=0.5):
        super(BiGRUSentiment, self).__init__()
        self.hidden_dim = hidden_dim
        self.use_gpu = use_gpu
        self.batch_size = batch_size
        self.dropout = dropout
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.gru = nn.GRU(input_size=embedding_dim, hidden_size=hidden_dim, bidirectional=True)
        self.hidden2label = nn.Linear(hidden_dim*2, label_size)
        self.hidden = self.init_hidden()

    def init_hidden(self):
        # first is the hidden h
        # second is the cell c
        if self.use_gpu:
            return (Variable(torch.zeros(2, self.batch_size, self.hidden_dim).cuda()))
        else:
            return (Variable(torch.zeros(2, self.batch_size, self.hidden_dim)))

    def forward(self, sentence):
        x = self.embeddings(sentence).view(len(sentence), self.batch_size, -1)
        gru_out, self.hidden = self.gru(x, self.hidden)
        y = self.hidden2label(torch.cat((gru_out[-1,:,:150], gru_out[0,:,150:]), 1))
        return y, gru_out, x
        #log_probs = F.log_softmax(y)
        #return log_probs

In [86]:
class GRUSentiment(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, label_size, use_gpu, batch_size, dropout=0.7):
        super(GRUSentiment, self).__init__()
        self.hidden_dim = hidden_dim
        self.use_gpu = use_gpu
        self.batch_size = batch_size
        self.dropout = dropout
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.gru = nn.GRU(input_size=embedding_dim, hidden_size=hidden_dim)
        self.hidden2label = nn.Linear(hidden_dim, label_size)
        self.hidden = self.init_hidden()

    def init_hidden(self):
        # first is the hidden h
        # second is the cell c
        if self.use_gpu:
            return (Variable(torch.zeros(1, self.batch_size, self.hidden_dim).cuda()))
        else:
            return (Variable(torch.zeros(1, self.batch_size, self.hidden_dim)))

    def forward(self, sentence):
        x = self.embeddings(sentence).view(len(sentence), self.batch_size, -1)
        gru_out, self.hidden = self.gru(x, self.hidden)
        y = self.hidden2label(gru_out[-1])
        return y, gru_out, x

In [87]:
    #     self.embedding = nn.Embedding(vocab_size, embedding_dim)
        
    #     #lstm layer
    #     self.lstm1 = nn.LSTM(embedding_dim, 
    #                        hidden_dim_1, 
    #                        num_layers=n_layers, 
    #                        bidirectional=bidirectional, 
    #                        dropout=0.25,
    #                        batch_first=True)
        
    #     self.lstm2 = nn.LSTM(hidden_dim_1,
    #                        hidden_dim_2, 
    #                        num_layers=n_layers, 
    #                        bidirectional=bidirectional, 
    #                        dropout=0.1)
        
    # # self.lstm_1 = nn.LSTM(self.nb_features, self.hidden_size_1, self.nb_layers_1, dropout=dropout)  
    # # self.lstm_2 = nn.LSTM(self.hidden_size_1, self.hidden_size_2, self.nb_layers_2, dropout=dropout)
        
    #     #dense layer

    #     # self.fc - nn.Linear(hidden_dim_2, 64)
    #     self.fc1 = nn.Linear(hidden_dim_2, 32)
        
    #     self. fc2 = nn.Linear(32, 1)

    #     #activation function
    #     self.act = nn.Sigmoid()


        # def forward(self, tweet):
        
        # #text = [batch size,sent_length]
        # # x = torch.FloatTensor(x)
        # embedded = torch.FloatTensor(self.embedding(tweet))
        # # flat_embedded = flattened = [item for sublist in embedded for item in sublist]
        # #embedded = [batch size, sent_len, emb dim]
      
        # #packed sequence
        # packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, embedding_dim, batch_first=True)
        
        # packed_output_1, (hidden_1, cell_1) = self.lstm1(packed_embedded)

        # output, _ = nn.utils.rnn.pad_packed_sequence(packed_output_1, batch_first=True)
        # # print(output.shape()) 
        # packed_output_2, (hidden_2, cell_2) = self.lstm2(output)



        # #hidden = [batch size, num layers * num directions,hid dim]
        # #cell = [batch size, num layers * num directions,hid dim]
        
        # #concat the final forward and backward hidden state
        # hidden = torch.cat((hidden_2[-2,:,:], hidden_2[-1,:,:]), dim = 1)
                
        # #hidden = [batch size, hid dim * num directions]
        # dense_outputs=torch.nn.functional.relu(self.fc1(hidden))

        # #Final activation function

        # outputs = torch.nn.functional.sigmoid(self.fc2(dense_outputs))
        # # outputs=self.act(dense_outputs)
        
        # return outputs

In [88]:
# class LSTMSentiment(nn.Module):

#     def __init__(self, embedding_dim, hidden_dim, vocab_size, label_size, use_gpu, batch_size, dropout=0.5):
#         super(LSTMSentiment, self).__init__()
#         self.hidden_dim = hidden_dim
#         self.use_gpu = use_gpu
#         self.batch_size = batch_size
#         #self.dropout = dropout
#         self.embeddings = nn.Embedding(vocab_size, embedding_dim)
#         self.dropout = nn.Dropout(dropout)
#         self.lstm = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_dim)
#         self.hidden2label = nn.Linear(hidden_dim, label_size)
#         self.hidden = self.init_hidden()

#     def init_hidden(self):
#         # first is the hidden h
#         # second is the cell c
#         if self.use_gpu:
#             return (Variable(torch.zeros(1, self.batch_size, self.hidden_dim).cuda()),
#                     Variable(torch.zeros(1, self.batch_size, self.hidden_dim).cuda()))
#         else:
#             return (Variable(torch.zeros(1, self.batch_size, self.hidden_dim)),
#                     Variable(torch.zeros(1, self.batch_size, self.hidden_dim)))

#     def forward(self, sentence):
#         x = self.embeddings(sentence).view(len(sentence), self.batch_size, -1)
#         lstm_out, self.hidden = self.lstm(x, self.hidden)
#         #lstm_out = self.dropout(lstm_out)
#         y = self.hidden2label(lstm_out[-1])
#         return y, lstm_out, x

In [89]:
class LSTMSentiment(nn.Module):

    def __init__(self, embedding_dim, hidden_dim, vocab_size, label_size, use_gpu, batch_size, hidden_dim_1, hidden_dim_2, dropout=0.5):
        super(LSTMSentiment, self).__init__()
        self.hidden_dim = hidden_dim
        self.use_gpu = use_gpu
        self.batch_size = batch_size
        #self.dropout = dropout
        self.embeddings = nn.Embedding(vocab_size, embedding_dim)
        self.dropout = nn.Dropout(dropout)
        self.hidden_dim_1 = hidden_dim_1
        self.hidden_dim_2 = hidden_dim_2
        # self.lstm1 = nn.LSTM(embedding_dim, hidden_dim_1, num_layers=n_layers, bidirectional=bidirectional, dropout=0.25, batch_first=True)
        # self.lstm2 = nn.LSTM(hidden_dim_1,hidden_dim_2, num_layers=n_layers, bidirectional=bidirectional, dropout=0.1)
        self.lstm1 = nn.LSTM(embedding_dim, hidden_dim_1, dropout=0.25)
        self.lstm2 = nn.LSTM(hidden_dim_1, hidden_dim_2, dropout=0.1)
        self.fc = nn.Linear(hidden_dim_2, 64)
        self.fc1 = nn.Linear(64, 32)
        # self.lstm = nn.LSTM(input_size=embedding_dim, hidden_size=hidden_dim)
        self.hidden2label = nn.Linear(32, label_size)
        self.hidden1 = self.init_hidden_1()
        self.hidden2 = self.init_hidden_2()
        self.act = nn.Sigmoid()

    def init_hidden_1(self):
        # first is the hidden h
        # second is the cell c
        if self.use_gpu:
            return (Variable(torch.zeros(1, self.batch_size, self.hidden_dim_1).cuda()),
                    Variable(torch.zeros(1, self.batch_size, self.hidden_dim_1).cuda()))
        else:
            return (Variable(torch.zeros(1, self.batch_size, self.hidden_dim_1)),
                    Variable(torch.zeros(1, self.batch_size, self.hidden_dim_1)))
            
    def init_hidden_2(self):
    # first is the hidden h
    # second is the cell c
        if self.use_gpu:
            return (Variable(torch.zeros(1, self.batch_size, self.hidden_dim_2).cuda()),
                    Variable(torch.zeros(1, self.batch_size, self.hidden_dim_2).cuda()))
        else:
            return (Variable(torch.zeros(1, self.batch_size, self.hidden_dim_2)),
                    Variable(torch.zeros(1, self.batch_size, self.hidden_dim_2)))
            
    def init_hidden_3(self):
    # first is the hidden h
    # second is the cell c
        if self.use_gpu:
            return (Variable(torch.zeros(1, 1, self.hidden_dim_1).cuda()),
                    Variable(torch.zeros(1, 1, self.hidden_dim_1).cuda()))
        else:
            return (Variable(torch.zeros(1, 1, self.hidden_dim_1)),
                    Variable(torch.zeros(1, 1, self.hidden_dim_1)))

    def forward(self, sentence):
        # self.dropout = 0.25
        print(sentence)
        x = self.embeddings(sentence).view(len(sentence), self.batch_size, -1)
        lstm_out_1, self.hidden = self.lstm1(x, self.hidden1)
        # lstm_out_1 = self.dropout(lstm_out_1)
        # self.dropout = 0.1
        lstm_out_2, self.hidden = self.lstm2(lstm_out_1, self.hidden2)
        # lstm_out_2 = self.dropout(lstm_out_2)
        # dense_outputs = torch.nn.functional.relu(self.fc1(lstm_out_2))
        # outputs = torch.nn.functional.sigmoid(self.fc2(dense_outputs))
        dense_outputs = torch.nn.functional.relu(self.fc(lstm_out_2))
        outputs = torch.nn.functional.sigmoid(self.fc1(dense_outputs))
        #lstm_out = self.dropout(lstm_out)
        y = self.hidden2label(outputs[-1])
        return y, outputs, dense_outputs
        # return y, lstm_out, x

In [90]:
!pwd

/content


In [91]:
# import torch
# import torch.nn as nn
from torchtext import data
import numpy as np
# from torch.autograd import Variable
from nltk.tokenize import word_tokenize
# import torch.nn.functional as F



def load_sst(text_field, label_field, batch_size):
    train, dev, test = data.TabularDataset.splits(path='/content/gdrive/My Drive/twitter misinfo/new_data/', train='train.tsv',
                                                  validation='dev.tsv', test='test.tsv', format='tsv',
                                                  fields=[('text', text_field), ('label', label_field)])
    text_field.build_vocab(train, dev, test)
    label_field.build_vocab(train, dev, test)
    train_iter, dev_iter, test_iter = data.BucketIterator.splits((train, dev, test),
                batch_sizes=(batch_size, len(dev), len(test)), sort_key=lambda x: len(x.text), repeat=False, device=-1) 
    return train_iter, dev_iter, test_iter

def sigmoid(x):
    return (1 / (1 + np.exp(-x)))


def bigru_attribution(model, text):
    # data for word-to-vec
    text_field = data.Field(lower=True)  # it is an object
    label_field = data.Field(sequential=False)   # it is also an object
    train_iter, dev_iter, test_iter = load_sst(text_field, label_field, 5)
    # word2vector
    word_to_idx = text_field.vocab.stoi  # example of word_to_idx: u'schools': 14512, len(word_to_idx) = 16190
    word_to_idx_dict = dict(word_to_idx)

    # processing text
    word_tokenize_list = word_tokenize(text)
    sentence = []
    for i in word_tokenize_list:
        sentence.append(word_to_idx_dict[i])
    sent = np.array(sentence)
    sent = Variable(torch.from_numpy(sent))
    sent = sent.cuda()
    length = len(word_tokenize_list)

    # model prediction
    best_model.batch_size = 1
    best_model.hidden = best_model.init_hidden()
    pred, hn, x = best_model(sent)
    hn = hn.cpu().data.numpy()
    x = x.cpu().data.numpy()
    pred = F.softmax(pred).cpu()
    pred_label = pred.data.max(1)[1].numpy()
    if pred_label[0] == 0:
        print ("prediction category: positive sentiment with confidence of " + str(pred.data.numpy()[0, 0]))# 0 is positive, and 1 is negative
    else:
        print ("prediction category: negative sentiment with confidence of " + str(pred.data.numpy()[0, 1]))

    # attribution for the prediction
    weights = best_model.gru.state_dict()

    _, W_iz, _ = np.split(weights['weight_ih_l0'].cpu().numpy(), 3, 0)
    _, W_hz, _ = np.split(weights['weight_hh_l0'].cpu().numpy(), 3, 0)
    _, b_z, _ = np.split(weights['bias_ih_l0'].cpu().numpy() + weights['bias_hh_l0'].cpu().numpy(), 3)

    _, W_iz_r, _ = np.split(weights['weight_ih_l0_reverse'].cpu().numpy(), 3, 0)
    _, W_hz_r, _ = np.split(weights['weight_hh_l0_reverse'].cpu().numpy(), 3, 0)
    _, b_z_r, _ = np.split(weights['bias_ih_l0_reverse'].cpu().numpy() + weights['bias_hh_l0_reverse'].cpu().numpy(), 3)



    z_dict = []
    z_dict.append(np.ones(150))
    for i in range(length-1):
        i = i + 1
        z = sigmoid(np.matmul(W_iz, x[i,0,:]) + np.matmul(W_hz, hn[i-1,0,:150]) + b_z)
        z_dict.append(z)
    alpha_dict = z_dict

    z_dict_reverse = []
    z_dict_reverse.append(np.ones(150))
    for i in range(length-1):
        i = length - 2 - i
        z = sigmoid(np.matmul(W_iz_r, x[i,0,:]) + np.matmul(W_hz_r, hn[i+1,0,150:]) + b_z_r)
        z_dict_reverse.append(z)
    z_dict_reverse = z_dict_reverse[::-1]
    alpha_dict_reverse = z_dict_reverse

    weights_linear = best_model.hidden2label.state_dict()
    W = weights_linear['weight'].cpu().numpy()
    b= weights_linear['bias'].cpu().numpy()

    
    target_class = pred_label
    score_dict = []
    for i in range(len(alpha_dict)):
        if i == 0:
            updating = hn[0,0,:150]
        else:
            updating = hn[i,0,:150] - alpha_dict[i] * hn[i-1,0,:150]
        forgetting = alpha_dict[0]
        for j in range(i+1, len(alpha_dict)):
            forgetting = forgetting*alpha_dict[j]

        if i == len(alpha_dict)-1:
            updating_reverse = hn[i,0,150:]
        else:
            updating_reverse = hn[i,0,150:] - alpha_dict_reverse[i] * hn[i+1,0,150:]
        forgetting_reverse = alpha_dict_reverse[-1]
        for j in range(i):
            forgetting_reverse = forgetting_reverse*alpha_dict_reverse[j]

        score = np.matmul( W[target_class], np.concatenate((updating * forgetting,updating_reverse*forgetting_reverse))) #+ b[target_class]
        score_dict.append(score[0])

    return word_tokenize_list, score_dict

In [92]:

def load_sst(text_field, label_field, batch_size):
    train, dev, test = data.TabularDataset.splits(path='/content/gdrive/My Drive/twitter misinfo/new_data/', train='train.tsv',
                                                  validation='dev.tsv', test='test.tsv', format='tsv',
                                                  fields=[('text', text_field), ('label', label_field)])
    text_field.build_vocab(train, dev, test)
    label_field.build_vocab(train, dev, test)
    train_iter, dev_iter, test_iter = data.BucketIterator.splits((train, dev, test),
                batch_sizes=(batch_size, len(dev), len(test)), sort_key=lambda x: len(x.text), repeat=False, device=-1) 
    return train_iter, dev_iter, test_iter

def sigmoid(x):
    return (1 / (1 + np.exp(-x)))



def gru_attribution(model, text):
    # data for word-to-vec
    text_field = data.Field(lower=True)  # it is an object
    label_field = data.Field(sequential=False)   # it is also an object
    train_iter, dev_iter, test_iter = load_sst(text_field, label_field, 5)
    # word2vector
    word_to_idx = text_field.vocab.stoi  # example of word_to_idx: u'schools': 14512, len(word_to_idx) = 16190
    word_to_idx_dict = dict(word_to_idx)

    # processing text
    word_tokenize_list = word_tokenize(text)
    sentence = []
    for i in word_tokenize_list:
        sentence.append(word_to_idx_dict[i])
    sent = np.array(sentence)
    sent = Variable(torch.from_numpy(sent))
    sent = sent.cuda()
    length = len(word_tokenize_list)

    # model prediction
    best_model.batch_size = 1
    best_model.hidden = best_model.init_hidden()
    pred, hn, x = best_model(sent)
    hn = hn.cpu().data.numpy()
    x = x.cpu().data.numpy()
    pred = F.softmax(pred).cpu()
    pred_label = pred.data.max(1)[1].numpy()  
    if pred_label[0] == 0:
        print ("prediction category: positive sentiment with confidence of " + str(pred.data.numpy()[0, 0]))# 0 is positive, and 1 is negative
    else:
        print ("prediction category: negative sentiment with confidence of " + str(pred.data.numpy()[0, 1]))


    # attribution for the prediction
    weights = best_model.gru.state_dict()
    _, W_iz, _ = np.split(weights['weight_ih_l0'].cpu().numpy(), 3, 0)
    _, W_hz, _ = np.split(weights['weight_hh_l0'].cpu().numpy(), 3, 0)
    _, b_z, _ = np.split(weights['bias_ih_l0'].cpu().numpy() + weights['bias_hh_l0'].cpu().numpy(), 3)

    z_dict = []
    z_dict.append(np.ones(150))
    for i in range(length-1):
        i = i + 1
        z = sigmoid(np.matmul(W_iz, x[i,0,:]) + np.matmul(W_hz, hn[i-1,0,:]) + b_z)
        z_dict.append(z)
    alpha_dict = z_dict
    weights_linear = best_model.hidden2label.state_dict()
    W = weights_linear['weight'].cpu().numpy()
    b= weights_linear['bias'].cpu().numpy()

    target_class = pred_label
    score_dict = []
    for i in range(len(alpha_dict)):
        if i == 0:
            updating = hn[0,0,:]
        else:
            updating = hn[i,0,:] - alpha_dict[i] * hn[i-1,0,:]
        forgetting = alpha_dict[0]
        for j in range(i+1, len(alpha_dict)):
            forgetting = forgetting*alpha_dict[j]
        score = np.matmul( W[target_class], updating * forgetting) #+ b[target_class]
        score_dict.append(score[0])
    return word_tokenize_list, score_dict



In [93]:

def load_sst(text_field, label_field, batch_size):
    train, dev, test = data.TabularDataset.splits(path='/content/gdrive/My Drive/twitter misinfo/new_data/', train='train.tsv',
                                                  validation='dev.tsv', test='test.tsv', format='tsv',
                                                  fields=[('text', text_field), ('label', label_field)])
    text_field.build_vocab(train, dev, test)
    label_field.build_vocab(train, dev, test)
    train_iter, dev_iter, test_iter = data.BucketIterator.splits((train, dev, test),
                batch_sizes=(batch_size, len(dev), len(test)), sort_key=lambda x: len(x.text), repeat=False, device=-1) 
    return train_iter, dev_iter, test_iter

def sigmoid(x):
    return (1 / (1 + np.exp(-x)))


def lstm_attribution(model, text):
    # data for word-to-vec
    text_field = data.Field(lower=True)  # it is an object
    label_field = data.Field(sequential=False)   # it is also an object
    train_iter, dev_iter, test_iter = load_sst(text_field, label_field, 5)
    # word2vector
    word_to_idx = text_field.vocab.stoi  # example of word_to_idx: u'schools': 14512, len(word_to_idx) = 16190
    word_to_idx_dict = dict(word_to_idx)

    # processing text
    word_tokenize_list = word_tokenize(text)
    sentence = []
    for i in word_tokenize_list:
        sentence.append(word_to_idx_dict[i])
    sent = np.array(sentence)
    sent = Variable(torch.from_numpy(sent))
    sent = sent.cuda()
    length = len(word_tokenize_list)

    # model prediction
    best_model.batch_size = 1
    best_model.hidden = best_model.init_hidden()
    pred, hn, x = best_model(sent)
    hn = hn.cpu().data.numpy()
    x = x.cpu().data.numpy()
    pred = F.softmax(pred).cpu()
    pred_label = pred.data.max(1)[1].numpy()  
    if pred_label[0] == 0:
        print ("prediction category: positive sentiment with confidence of " + str(pred.data.numpy()[0, 0]))# 0 is positive, and 1 is negative
    else:
        print ("prediction category: negative sentiment with confidence of " + str(pred.data.numpy()[0, 1]))

    # attribution for the prediction
    weights = best_model.lstm.state_dict()
    W_ii, W_if, W_ig, W_io = np.split(weights['weight_ih_l0'].cpu().numpy(), 4, 0)
    W_hi, W_hf, W_hg, W_ho = np.split(weights['weight_hh_l0'].cpu().numpy(), 4, 0)
    b_i, b_f, b_g, b_o = np.split(weights['bias_ih_l0'].cpu().numpy() + weights['bias_hh_l0'].cpu().numpy(), 4)


    z_dict = []
    z_dict.append(np.ones(150))
    for i in range(length-1):
        i = i + 1
        z = sigmoid(np.matmul(W_if, x[i,0,:]) + np.matmul(W_hf, hn[i-1,0,:]) + b_f)
        z_dict.append(z)

    o_dict = []
    for i in range(length):
        if i == 0:
            o = sigmoid(np.matmul(W_io, x[i,0,:]) + b_o)
        else:
            o = sigmoid(np.matmul(W_io, x[i,0,:]) + np.matmul(W_ho, hn[i-1,0,:]) + b_o)
        o_dict.append(o)


    alpha_dict = []
    for i in range(len(z_dict)):
        if i == 0:
            alpha_dict.append(z_dict[0])
        else:
            alpha_dict.append(z_dict[i] * o_dict[i] / o_dict[i-1])

    weights_linear = best_model.hidden2label.state_dict()
    W = weights_linear['weight'].cpu().numpy()
    b= weights_linear['bias'].cpu().numpy()
    target_class = pred_label


    score_dict = []
    for i in range(len(alpha_dict)):
        if i == 0:
            updating = hn[0,0,:]
        else:
            updating = hn[i,0,:] - alpha_dict[i] * hn[i-1,0,:]
        forgetting = alpha_dict[0]
        for j in range(i+1, len(alpha_dict)):
            forgetting = forgetting*alpha_dict[j]
        score = np.matmul( W[target_class], updating * forgetting) 
        score_dict.append(score[0])
    return word_tokenize_list, score_dict

In [94]:
# import torch
# import torch.nn as nn
from torch import optim
import time, random
import os
from tqdm import tqdm
# from lstm import LSTMSentiment
# from gru import GRUSentiment
# from bigru import BiGRUSentiment
from torchtext import data
import numpy as np
import argparse
# from torch.autograd import Variable
# import torch.nn.functional as F

torch.set_num_threads(8)
torch.manual_seed(1)
random.seed(1)


def load_bin_vec(fname, vocab):
    """
    Loads 300x1 word vecs from Google (Mikolov) word2vec
    """
    word_vecs = {}
    with open(fname, "rb") as f:
        header = f.readline()
        vocab_size, layer1_size = map(int, header.split())
        binary_len = np.dtype('float32').itemsize * layer1_size
        for line in range(vocab_size):
            word = []
            while True:
                ch = f.read(1).decode('latin-1')
                if ch == ' ':
                    word = ''.join(word)
                    break
                if ch != '\n':
                    word.append(ch)
            if word in vocab:
               word_vecs[word] = np.fromstring(f.read(binary_len), dtype='float32')
            else:
                f.read(binary_len)
    return word_vecs


def get_accuracy(truth, pred):
    assert len(truth) == len(pred)
    right = 0
    for i in range(len(truth)):
        if truth[i] == pred[i]:
            right += 1.0
    return right / len(truth)


def train_epoch_progress(model, train_iter, loss_function, optimizer, text_field, label_field, epoch):
    model.train()
    avg_loss = 0.0
    truth_res = []
    pred_res = []
    count = 0
    for batch in tqdm(train_iter, desc='Train epoch '+str(epoch+1)):
        sent, label = batch.text, batch.label
        label.data.sub_(1)
        truth_res += list(label.data)
        model.batch_size = len(label.data)
        # print("BATCH_SIZE = ", model.batch_size)
        # model.hidden = model.init_hidden_1()
        # model.batch_size = 1
        model.hidden = model.init_hidden_1()
        print(sent)
        sent = sent.cuda()
        pred, _, _ = model(sent)
        pred = F.log_softmax(pred)
        pred = pred.cpu()
        pred_label = pred.data.max(1)[1].numpy()
        pred_res += [x for x in pred_label]
        model.zero_grad()
        loss = loss_function(pred, label)
        # avg_loss += loss.data[0]
        avg_loss += loss.data
        count += 1
        loss.backward()
        optimizer.step()
    avg_loss /= len(train_iter)
    acc = get_accuracy(truth_res, pred_res)
    return avg_loss, acc


def evaluate(model, data, loss_function, name):
    model.eval()
    avg_loss = 0.0
    truth_res = []
    pred_res = []
    for batch in data:
        sent, label = batch.text, batch.label
        label.data.sub_(1)
        truth_res += list(label.data)
        model.batch_size = len(label.data)
        # model.hidden = model.init_hidden_1()
        model.hidden = model.init_hidden_1()
        sent = sent.cuda()
        pred, _, _ = model(sent)
        pred = F.log_softmax(pred)
        pred = pred.cpu()
        pred_label = pred.data.max(1)[1].numpy()
        pred_res += [x for x in pred_label]
        loss = loss_function(pred, label)
        # avg_loss += loss.data[0]
        avg_loss += loss.data
    avg_loss /= len(data)
    acc = get_accuracy(truth_res, pred_res)
    print(name + ': loss %.2f acc %.1f' % (avg_loss, acc*100))
    return acc


def load_sst(text_field, label_field, batch_size):
    train, dev, test = data.TabularDataset.splits(path='/content/gdrive/My Drive/twitter misinfo/prepro_data/', train='train.tsv',
                                                  validation='dev.tsv', test='test.tsv', format='tsv',
                                                  fields=[('text', text_field), ('label', label_field)])
    text_field.build_vocab(train, dev, test)
    label_field.build_vocab(train, dev, test)
    train_iter, dev_iter, test_iter = data.BucketIterator.splits((train, dev, test),
                batch_sizes=(batch_size, len(dev), len(test)), sort_key=lambda x: len(x.text), repeat=False, device=-1)
    ## for GPU run
#     train_iter, dev_iter, test_iter = data.BucketIterator.splits((train, dev, test),
#                 batch_sizes=(batch_size, len(dev), len(test)), sort_key=lambda x: len(x.text), repeat=False, device=None)
    return train_iter, dev_iter, test_iter



# args = argparse.ArgumentParser()
# args.add_argument('--m', dest='model', default='lstm', help='specify the mode to use (default: lstm)')
# args = args.parse_args()

def runModel( modelName='lstm'):
    EPOCHS = 20
    USE_GPU = torch.cuda.is_available()
    EMBEDDING_DIM = 300
    HIDDEN_DIM = 150

    BATCH_SIZE = 5
    timestamp = str(int(time.time()))
    best_dev_acc = 0.0

    hid_1 = 300
    hid_2 = 128


    # data is from torchtext.data, it is an object
    text_field = data.Field(lower=True)
    label_field = data.Field(sequential=False)
    train_iter, dev_iter, test_iter = load_sst(text_field, label_field, BATCH_SIZE)
    print(text_field, label_field)


    if modelName == 'lstm':
        print('model is lstm')
        model = LSTMSentiment(embedding_dim=EMBEDDING_DIM, hidden_dim=HIDDEN_DIM, vocab_size=len(text_field.vocab), label_size=len(label_field.vocab)-1, use_gpu=USE_GPU, batch_size=BATCH_SIZE, hidden_dim_1 = hid_1, hidden_dim_2 = hid_2)

    if modelName == 'gru':
        print('model is gru')
        model = GRUSentiment(embedding_dim=EMBEDDING_DIM, hidden_dim=HIDDEN_DIM, vocab_size=len(text_field.vocab), label_size=len(label_field.vocab)-1,\
                            use_gpu=USE_GPU, batch_size=BATCH_SIZE)

    if modelName == 'bigru':
        print('model is bigru')
        model = BiGRUSentiment(embedding_dim=EMBEDDING_DIM, hidden_dim=HIDDEN_DIM, vocab_size=len(text_field.vocab), label_size=len(label_field.vocab)-1,\
                            use_gpu=USE_GPU, batch_size=BATCH_SIZE)

    if USE_GPU:
        print('use cuda')
        model = model.cuda()
        # .deatch().cpu().clone().numpy()


    print('Load word embeddings...')
    # word2vector
    word_to_idx = text_field.vocab.stoi
    # example of word_to_idx: u'schools': 14512, u'mimicking': 12930
    # len(word_to_idx) = 16190
    pretrained_embeddings = np.random.uniform(-0.25, 0.25, (len(text_field.vocab), 300))
    # pretrained_embeddings.shape = (16190, 300)

    pretrained_embeddings[0] = 0 #set all pretrained_embeddings[0] to 0
    word2vec = load_bin_vec('/content/gdrive/My Drive/twitter misinfo/news_vec/GoogleNews-vectors-negative300.bin', word_to_idx)
    for word, vector in word2vec.items():
        pretrained_embeddings[word_to_idx[word]-1] = vector
    # the embedding layer of this model is using the pretrained embeddings
    model.embeddings.weight.data.copy_(torch.from_numpy(pretrained_embeddings))
    # model.embeddings.weight.data = text_field.vocab.vectors
    # model.embeddings.embed.weight.requires_grad = False


    best_model = model
    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    loss_function = nn.NLLLoss()


    print('Training...')
    if modelName == 'gru':
        out_dir = "models/gru"
    elif modelName == 'lstm':
        out_dir = "models/lstm"
    else:
        out_dir = "models/bigru"

    #out_dir = os.path.abspath(os.path.join(os.path.curdir, "runs", timestamp))
    print("Writing to {}\n".format(out_dir))
    if not os.path.exists(out_dir):
        os.makedirs(out_dir)
    for epoch in range(EPOCHS):
        avg_loss, acc = train_epoch_progress(model, train_iter, loss_function, optimizer, text_field, label_field, epoch)
        tqdm.write('Train: loss %.2f acc %.1f' % (avg_loss, acc*100))
        dev_acc = evaluate(model, dev_iter, loss_function, 'Dev')
        if dev_acc > best_dev_acc:
            if best_dev_acc > 0:
                os.system('rm '+ out_dir + '/best_model' + '.pkl')
            best_dev_acc = dev_acc
            best_model = model
            torch.save(best_model, out_dir + '/best_model' + '.pkl')
            # evaluate on test with the best dev performance model
            test_acc = evaluate(best_model, test_iter, loss_function, 'Test')


    return test_iter, loss_function

In [95]:
# !wget https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit

In [96]:
# !wget -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"

In [97]:
# import zipfile


# zip_ref = zipfile.ZipFile("/content/GoogleNews-vectors-negative300.bin.gz", 'r')
# zip_ref.extractall("/content/gdrive/My Drive/twitter misinfo/REATData/")
# zip_ref.close()

# !tar -xvf  '/content/GoogleNews-vectors-negative300.bin.gz' -C '/content/gdrive/My Drive/twitter misinfo/REATData/'

In [98]:
# !unzip '/content/GoogleNews-vectors-negative300.bin.gz' -d '/content/gdrive/My Drive/twitter misinfo/REATData/'

In [99]:
# !gunzip '/content/GoogleNews-vectors-negative300.bin.gz'

In [100]:
!mv '/content/GoogleNews-vectors-negative300.bin' '/content/gdrive/My Drive/twitter misinfo/news_vec/'

mv: cannot stat '/content/GoogleNews-vectors-negative300.bin': No such file or directory


In [101]:
import torch
print(torch.__version__)


1.5.1+cu101


In [102]:
model_name = 'lstm'

if model_name == 'gru':
    out_dir = "models/gru"
elif model_name == 'lstm':
    out_dir = "models/lstm"
else:
    out_dir = "models/bigru"

test_iter, loss_function = runModel(modelName = model_name)


best_model = torch.load(out_dir + '/best_model' + '.pkl')
test_acc = evaluate(best_model, test_iter, loss_function, 'Final Test')

The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.
The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.
The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.
/usr/local/lib/python3.6/dist-packages/torch/nn/modules/rnn.py:50: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.25 and num_layers=1
  "num_layers={}".format(dropout, num_layers))
/usr/local/lib/python3.6/dist-packages/torch/nn/modules/rnn.py:50: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.1 an

<torchtext.data.field.Field object at 0x7f2146760320> <torchtext.data.field.Field object at 0x7f2146760dd8>
model is lstm
use cuda
Load word embeddings...


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:40: DeprecationWarning: The binary mode of fromstring is deprecated, as it behaves surprisingly on unicode inputs. Use frombuffer instead


Train epoch 1:   0%|          | 0/2800 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/torch/nn/functional.py:1569: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:73: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


Train epoch 1:   0%|          | 9/2800 [00:00<00:31, 88.89it/s]

Training...
Writing to models/lstm

tensor([[13549, 15715, 12378, 19663, 11905]])
tensor([[13549, 15715, 12378, 19663, 11905]], device='cuda:0')
tensor([[10045, 13369, 15284, 14966, 20060]])
tensor([[10045, 13369, 15284, 14966, 20060]], device='cuda:0')
tensor([[20210, 15590, 17764, 16784, 15475]])
tensor([[20210, 15590, 17764, 16784, 15475]], device='cuda:0')
tensor([[ 9584,  3692, 13299,  9372, 10930]])
tensor([[ 9584,  3692, 13299,  9372, 10930]], device='cuda:0')
tensor([[12892, 16402, 15900, 15924,  3939]])
tensor([[12892, 16402, 15900, 15924,  3939]], device='cuda:0')
tensor([[19444, 14461, 10733, 18211, 15495]])
tensor([[19444, 14461, 10733, 18211, 15495]], device='cuda:0')
tensor([[12475,  5213, 13963,  5072, 16884]])
tensor([[12475,  5213, 13963,  5072, 16884]], device='cuda:0')
tensor([[12514, 13834, 19777, 17946,  4964]])
tensor([[12514, 13834, 19777, 17946,  4964]], device='cuda:0')
tensor([[18833, 15982, 19900, 16137, 17918]])
tensor([[18833, 15982, 19900, 16137, 17918]], 



Train epoch 1:   1%|          | 21/2800 [00:00<00:29, 95.44it/s]

Train epoch 1:   1%|          | 33/2800 [00:00<00:27, 100.20it/s]

...



Train epoch 1:  96%|█████████▌| 2677/2800 [00:23<00:01, 113.23it/s]

Train epoch 1:  96%|█████████▌| 2689/2800 [00:24<00:00, 113.36it/s]

tensor([[11235, 16715, 19161, 14960, 11812]])
tensor([[11235, 16715, 19161, 14960, 11812]], device='cuda:0')
tensor([[20056, 17128, 15019, 11345, 13951]])
tensor([[20056, 17128, 15019, 11345, 13951]], device='cuda:0')
tensor([[12337, 17677, 17847, 10416, 10898]])
tensor([[12337, 17677, 17847, 10416, 10898]], device='cuda:0')
tensor([[11486, 15004, 13783,  9342,  3327]])
tensor([[11486, 15004, 13783,  9342,  3327]], device='cuda:0')
tensor([[ 5512, 13806, 17716, 17986, 18835]])
tensor([[ 5512, 13806, 17716, 17986, 18835]], device='cuda:0')
tensor([[ 3180, 20273, 16026, 11754, 10145]])
tensor([[ 3180, 20273, 16026, 11754, 10145]], device='cuda:0')
tensor([[18024,  5068, 17777, 19379, 12204]])
tensor([[18024,  5068, 17777, 19379, 12204]], device='cuda:0')
tensor([[16531, 18324, 14144, 11646, 17271]])
tensor([[16531, 18324, 14144, 11646, 17271]], device='cuda:0')
tensor([[ 3843, 18534, 18088, 15555, 13506]])
tensor([[ 3843, 18534, 18088, 15555, 13506]], device='cuda:0')
tensor([[ 5158,  42



Train epoch 1:  96%|█████████▋| 2701/2800 [00:24<00:00, 113.29it/s]

tensor([[ 5862, 16844, 12319,  3912, 10007]])
tensor([[ 5862, 16844, 12319,  3912, 10007]], device='cuda:0')
tensor([[ 5005,  4610, 16281, 17470,  4754]])
tensor([[ 5005,  4610, 16281, 17470,  4754]], device='cuda:0')
tensor([[ 9174,  4000, 11347, 19933, 11009]])
tensor([[ 9174,  4000, 11347, 19933, 11009]], device='cuda:0')
tensor([[ 3128,  9518, 20248,  4209, 19799]])
tensor([[ 3128,  9518, 20248,  4209, 19799]], device='cuda:0')
tensor([[18051,  3158, 15605,  5655,  4875]])
tensor([[18051,  3158, 15605,  5655,  4875]], device='cuda:0')
tensor([[ 4765, 11466, 14499, 12559,  5083]])
tensor([[ 4765, 11466, 14499, 12559,  5083]], device='cuda:0')
tensor([[ 3095, 11924, 19478, 19801, 19775]])
tensor([[ 3095, 11924, 19478, 19801, 19775]], device='cuda:0')
tensor([[14604, 17845, 11454,  5840,  9519]])
tensor([[14604, 17845, 11454,  5840,  9519]], device='cuda:0')
tensor([[19688,  4191,  3739, 16494, 14589]])
tensor([[19688,  4191,  3739, 16494, 14589]], device='cuda:0')
tensor([[ 4953,  38

RuntimeError: ignored

In [ ]:
model_name = 'gru'

if model_name == 'gru':
    out_dir = "models/gru"
elif model_name == 'lstm':
    out_dir = "models/lstm"
else:
    out_dir = "models/bigru"

test_iter, loss_function = runModel(modelName = model_name)


best_model = torch.load(out_dir + '/best_model' + '.pkl')
test_acc = evaluate(best_model, test_iter, loss_function, 'Final Test')

In [ ]:
model_name = 'bigru'

if model_name == 'gru':
    out_dir = "models/gru"
elif model_name == 'lstm':
    out_dir = "models/lstm"
else:
    out_dir = "models/bigru"

test_iter, loss_function = runModel(modelName = model_name)


best_model = torch.load(out_dir + '/best_model' + '.pkl')
test_acc = evaluate(best_model, test_iter, loss_function, 'Final Test')

In [ ]:
import nltk

nltk.download('punkt')

In [ ]:
# testing text REAT GRU
# text = "think asked thing week two ago getting accurate death  know fact its opinion death obvious and even are pile still living"
text = " fight scenes fun grows tedious"
# text = "the story may be new, but it does not serve lots of laughs"
best_model = torch.load('models/gru/best_model.pkl')
word_tokenize_list, score_dict = gru_attribution(best_model, text) 


for i in range(len(score_dict)):
    print(word_tokenize_list[i], score_dict[i])

In [ ]:
best_model

In [ ]:
# device = torch.device('cpu')

In [ ]:
# testing text REAT LSTM
text = " fight scenes  fun  grows tedious"
# text = "the story may be new, but it does not serve lots of laughs"
best_model = torch.load('models/lstm/best_model.pkl')
word_tokenize_list, score_dict = lstm_attribution(best_model, text)


for i in range(len(score_dict)):
    print(word_tokenize_list[i], score_dict[i])

In [ ]:
# testing text REAT BiGRU
# text = "the fight scenes are fun but it grows tedious"
# text = "the story may be new, but it does not serve lots of laughs"
text = "many people rely transit get food healthcare essential businesses using transit time take  survey help advocate safe reliable transit move us past hard times"
best_model = torch.load('models/bigru/best_model.pkl')

word_tokenize_list, score_dict = bigru_attribution(best_model, text) 


for i in range(len(score_dict)):
    print(word_tokenize_list[i], ' = ', score_dict[i])

In [ ]:
# !pip install termcolor


In [ ]:
# new_dictionary = dict(zip(word_tokenize_list, score_dict))
# # print(new_dictionary)

# for i in new_dictionary:
#     print(new_dictionary[i])

In [ ]:
# from termcolor import colored
# text='the story may be new, but it does not serve lots of laughs'
# l1 = text.split()
# result = " ".join(colored(t,'white', 'on_green') if t in l1 else t for t in text.lower().split())
# print(result)

# # word_tokenize_list, score_dict = bigru_attribution(best_model, text) 


# # for i in range(len(score_dict)):
# #     print(word_tokenize_list[i], ' = ', score_dict[i])

# # for i in range(len(score_dict)):
# #     if score_dict[t] > 0:
# #         result = " ".join(colored(t, 'white', 'on_green')if t in l1 else t for t in text.lower().split())
# #     else:
# #         result = " ".join(colored(t, 'white', 'on_red')if t in l1 else t for t in text.lower().split())

# for i in new_dictionary:
#     if new_dictionary[i] > 0:
#         result = " ".join(colored(i, 'white', 'on_green'))
#     else:
#         result = " ".join(colored(i, 'white', 'on_red'))

# print(result)

In [ ]:
# !pip install colorama

In [ ]:
# import colorama
# from termcolor import *

# colorama.init()
# cprint(colored('Hello, World!', 'red', 'on_grey', ['blink']))
# colored('Hello, World!', 'green')

In [ ]:
# text_dict = dict(zip(word_tokenize_list, score_dict))
# print(len(text_dict))
# text_dict


In [ ]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt


# # def green_color_func(word, font_size, position, orientation, random_state=None,
# #                     **kwargs):
# #     return "hsl(0, 0%%, %d%%)" % random.randint(60, 100)

# def new_color_func(default=text_dict, *args, **kwargs):
#     for key, value in text_dict.items():
#         if value > 0:
#             color = '#ff0000' # red
#         elif value < 0:
#             color = '#00ff00' # green

#         return color
#     # if word in ['hello']:
#     #     color = '#ff0000' # red
#     # elif word in ['hi']:
#     #     color = '#00ff00' # green
#     # elif word in ['im', 'good']:
#     #     color = '#0000ff' # blue
#     # elif word in ['there']:
#     #     color = 'ffa500' # orange
#     # else:
#     #     color = '#000000' # black
#     # return color

# # wc = Wordcloud(..., color_func=color_func, ...)
# # wc.generate(word_string) 

# wc = WordCloud(margin=10,
#                random_state=1).generate(text)
# # store default colored image
# default_colors = wc.to_array()
# plt.title("Custom colors")
# plt.imshow(wc.recolor(color_func=new_color_func, random_state=3),interpolation="bilinear")
# # wc.to_file("a_new_hope.png")
# plt.axis("off")
# plt.figure()
# plt.title("Default colors")
# plt.imshow(default_colors, interpolation="bilinear")
# plt.axis("off")
# plt.show()

In [ ]:
import seaborn as sns
import pandas as pd
import numpy as np

hm_df = pd.DataFrame({'words' : word_tokenize_list, 'sentiment_score' : score_dict})

hm_df

In [ ]:
def ceildiv(a, b):
    return -(-a // b)

ceildiv(18, 5)

# def div(a, b):
#     if ((a/b) - int(a/b)) > 0.5 and ((a/b) - int(a/b)) < 1.5:
#         return int(a/b) + 1
#     else:
#         return int(a/b)

# div(18, 5)

In [ ]:
# hm_df = hm_df.sort_values('sentiment_score', ascending=False)
print(len(hm_df))
hm_df['col'] = pd.Series(np.random.randn(len(hm_df)), index=hm_df.index)
hm_df['row'] = 1
hm_df


In [ ]:
for i in range(len(hm_df)):
    hm_df['col'][i] = int(i)

hm_df

In [ ]:
word = ((np.asarray(hm_df['words'])).reshape(len(hm_df), 1))
sen_score = ((np.asarray(hm_df['sentiment_score'])).reshape(len(hm_df), 1))

print(word)
print(sen_score)

In [ ]:
result = hm_df.pivot(index = 'col', columns = 'row', values = 'sentiment_score')
print(result)

In [ ]:
labels = (np.asarray(["{0} \n {1:.6f}".format(word, value) for word,value in zip(word.flatten(), sen_score.flatten())]) ).reshape(len(hm_df), 1)

In [ ]:
fig, ax = plt.subplots(figsize = (5,14))
title = "Word Sentiment score heatmap"
plt.title(title, fontsize=18)
ttl = ax.title
ttl.set_position([0.5,1.05])
ax.set_xticks([])
ax.set_yticks([])
ax.axis('off')

sns.heatmap(result, annot=labels, fmt = "", cmap = 'RdYlGn', linewidths=0.30, ax=ax)
plt.show()